# Install library and Mount google drive

In [3]:
!pip install pythainlp
!pip install sklearn-pycrfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Connect with Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import library

In [5]:
import os
import datetime

import pandas as pd 
import numpy as np
from numpy import array

from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Helper function

In [6]:
def group_date(date):
  date = date.strip()
  d, m, y = date.split('/')
  year, week, weekday = datetime.date(int(y), int(m), int(d)).isocalendar()
  return str(year) + str(week).zfill(2)

# Preprocessing Soybean News

>## Import Soybean News

In [7]:
soybean = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/News.csv')
soybean.sort_values(by=['url'], inplace = True, ignore_index=True)
soybean['product'] = 'soybean'

soybean

,url,title,ถั่วเหลือง,product
0,100739,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2  4 ม...,ราคาทรงตัว ความต้องการใช้ผลผลิตกากถั่วเหลืองใน...,soybean
1,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ราคาลดลง ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอ...,soybean
2,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,ราคาทรงตัว สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั...,soybean
3,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,ราคาทรงตัว สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั...,soybean
4,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ราคาทรงตัว ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่าง...,soybean
...,...,...,...,...
354,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,ราคาเพิ่มขึ้น กากถั่วเหลืองจากเมล็ดถั่วเหลืองน...,soybean
355,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,ราคาเพิ่มขึ้น กากถั่วเหลืองจากเมล็ดถั่วเหลืองน...,soybean
356,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,ราคาทรงตัว กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข...,soybean
357,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,ราคาทรงตัว กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข...,soybean


>## Price classification

In [8]:
def extract_news_price_and_trend(text):
  y_position = text.find(' ')
  trend_position = text.find('แนวโน้ม')
  if not (trend_position == -1 or y_position == -1):
    remain_text = text[trend_position+7:]
    while (True):
      temp = remain_text.find('แนวโน้ม')
      if temp == -1:
        break
      else:
        trend_position += temp + 7
        remain_text = remain_text[temp+7:]
    trend = text[trend_position:]
    return text[:y_position], text[y_position+1:trend_position], trend.split(":")[1].replace(' ','')

temp_df = pd.DataFrame(soybean['ถั่วเหลือง'].apply(extract_news_price_and_trend).tolist(), columns=['news_price', 'ถั่วเหลือง', 'news_price_trend'])
soybean['ถั่วเหลือง'] = temp_df['ถั่วเหลือง']
soybean['news_price'] = temp_df['news_price']
soybean['news_price_trend'] = temp_df['news_price_trend']

y_keywords = {'ราคาทรงตัว' : 'sideway',
              'ราคาเพิ่มขึ้น' : 'bullish',
              'ราคาอ่อนตัว' : 'bearish',
              'ราคาลดลง' : 'bearish',
              'ราคาเพิ่ม' : 'bullish',
              'ราคาปรับขึ้น' : 'bullish',
              'ราคาปรับลด' : 'bearish'
              }

trend_keywords = {'ขึ้น' : 'bullish',
                  'ลง' : 'bearish',
                  'อ่อนตัว' : 'bearish',
                  'ทรง' : 'sideway',
                  'ยืน' : 'sideway',
                  'แข็งตัว' : 'sideway'
                  }

def re_format(text, keywords):
  for key in keywords.keys():
    if text.find(key) != -1:
      return keywords[key]

soybean['news_price'] = soybean['news_price'].apply(re_format, keywords=y_keywords)
soybean['news_price_trend'] = soybean['news_price_trend'].apply(re_format, keywords=trend_keywords)

def clssification_y(y):
  if y == 'bullish':
    return str(1)
  elif y == 'sideway':
    return str(0)
  else :
    return str(-1)

soybean['news_price'] = soybean['news_price'].apply(clssification_y)
soybean['news_price_trend'] = soybean['news_price_trend'].apply(clssification_y)

soybean.drop(index=soybean.index[0], axis=0, inplace=True)
soybean.reset_index(drop=True, inplace=True)

soybean.drop(['news_price_trend'], axis=1, inplace=True)
soybean

,url,title,ถั่วเหลือง,product,news_price
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0
...,...,...,...,...,...
353,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1
354,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1
355,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0
356,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0


>## Group date

In [9]:
soybean['week'] = soybean['title'].apply(str.replace, args=('สรุปภาวะสินค้าเกษตรประจำสัปดาห์',''))

months = {'มกราคม' : '/1/',
          'กุมภาพันธ์' : '/2/',
          'มีนาคม' : '/3/',
          'เมษายน' : '/4/',
          'พฤษภาคม' : '/5/',
          'มิถุนายน' : '/6/',
          'กรกฎาคม' : '/7/',
          'สิงหาคม' : '/8/',
          'กันยายน' : '/9/',
          'ตุลาคม' : '/10/',
          'พฤศจิกายน' : '/11/',
          'ธันวาคม' : '/12/'}

def extract_date(data):
  data = data.replace(' ','').replace('วันที่','').strip()
  for key, value in months.items():
    data = data.replace(key,value)
  years = int(data[-4:])-543
  data = data[:-4]+str(years)
  data = data.split('-')[-1]
  if data.startswith('0'):
    return data[1:]
  return data

soybean['lastday'] = soybean['week'].apply(extract_date)
soybean

,url,title,ถั่วเหลือง,product,news_price,week,lastday
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1,วันที่ 9 -13 กุมภาพันธ์ 2558,13/2/2015
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 16-20 กุมภาพันธ์ 2558,20/2/2015
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 23 - 27กุมภาพันธ์2558,27/2/2015
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 2 - 6 มีนาคม 2558,6/3/2015
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 9-13 มีนาคม 2558,13/3/2015
...,...,...,...,...,...,...,...
353,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 3-7 มกราคม 2565,7/1/2022
354,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 10-14 มกราคม 2565,14/1/2022
355,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 17-21 มกราคม 2565,21/1/2022
356,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 24-28 มกราคม 2565,28/1/2022


In [10]:
soybean['lastday'] = soybean['lastday'].apply(group_date)
soybean = soybean.rename(columns={'lastday': 'dates'})

>## Word token and remove stop word

In [11]:
def word_token(text):
  # return ['\'' + word + '\'' for word in word_tokenize(text)]
  return ' '.join(word_tokenize(text))
  
soybean['whole_word_token'] = soybean['ถั่วเหลือง'].apply(word_token)

soybean

,url,title,ถั่วเหลือง,product,news_price,week,dates,whole_word_token
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 16-20 กุมภาพันธ์ 2558,201508,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 23 - 27กุมภาพันธ์2558,201509,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 2 - 6 มีนาคม 2558,201510,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 9-13 มีนาคม 2558,201511,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...
...,...,...,...,...,...,...,...,...
353,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 3-7 มกราคม 2565,202201,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
354,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 10-14 มกราคม 2565,202202,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
355,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 17-21 มกราคม 2565,202203,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
356,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 24-28 มกราคม 2565,202204,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...


In [12]:
pd.DataFrame(soybean['whole_word_token'])

,whole_word_token
0,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...
1,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...
2,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...
3,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...
4,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...
...,...
353,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
354,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
355,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...
356,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...


In [13]:
soybean['whole_word_token'].loc[0]

'ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกับ ความต้องการ ใช้ ที่ มี อย่าง ต่อเนื่อง   ขณะที่ บราซิล ใกล้ เข้าสู่ ฤดูกาล เก็บเกี่ยว ผลผลิต ถั่วเหลือง ที่จะ มี ขึ้น ใน เดือน มีนาคม นี้   ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ลดลง จาก กิโลกรัม ละ   18.95   บาท   เป็น กิโลกรัม ละ   18.65   บาท   ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก   ประจำ วันที่   12   กุมภาพันธ์   2557   รอบ ส่งมอบ เดือน มีนาคม   ราคา อยู่ ที่   330.50   ดอล ลา รส ห รัฐ / ชอร์ต ตัน   ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม   ราคา อยู่ ที่   983.75   เซนต์ / บุช เชล   ซึ่ง เป็น รา คาที่ เพิ่มขึ้น   เนื่องจาก บราซิล มี การคาดการณ์ ปี นี้ ผลผลิต ถั่วเหลือง มี น้อย  '

>## Word  Representaive

>>### Count Vec aka Bag of word

>>>#### Whole

In [14]:
cvec = CountVectorizer()
c_vec = cvec.fit_transform(soybean['whole_word_token'])

In [15]:
x_c_vec = pd.DataFrame(c_vec.toarray(), columns=cvec.get_feature_names(), index=soybean['whole_word_token'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
x_c_vec.head()

,00,000,001,003,005,006,007,011,012,014,...,ได,ไตรมาส,ไบโอด,ไป,ไม,ไร,ไว,ไวร,ไอ,ไอโอวา
whole_word_token,,,,,,,,,,,,,,,,,,,,,
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกับ ความต้องการ ใช้ ที่ มี อย่าง ต่อเนื่อง ขณะที่ บราซิล ใกล้ เข้าสู่ ฤดูกาล เก็บเกี่ยว ผลผลิต ถั่วเหลือง ที่จะ มี ขึ้น ใน เดือน มีนาคม นี้ ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ลดลง จาก กิโลกรัม ละ 18.95 บาท เป็น กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 12 กุมภาพันธ์ 2557 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 330.50 ดอล ลา รส ห รัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 983.75 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ เพิ่มขึ้น เนื่องจาก บราซิล มี การคาดการณ์ ปี นี้ ผลผลิต ถั่วเหลือง มี น้อย,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
"สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหลือง ยังคง ทรงตัว ขณะที่ ราคา ผลผลิต ถั่วเหลือง ในประเทศ ขึ้นอยู่กับ ราคา ของตลาด ชิคาโก เป็นหลัก โดย กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า สัปดาห์ นี้ ราคา ไม่ เปลี่ยนแปลง จาก สัปดาห์ ที่ผ่านมา อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 19 กุมภาพันธ์ 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 347.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 1,007.25 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก พื้นที่ ใน การ เพาะปลูก ถั่วเหลือง น้อยกว่า ที่ คาดการณ์ ไว้ ก่อนหน้านี้",0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
"สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหลือง ยังคง ทรงตัว ขณะที่ ราคา ผลผลิต ถั่วเหลือง ในประเทศ ขึ้นอยู่กับ ราคา ของตลาด ชิคาโก เป็นหลัก โดย กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า สัปดาห์ นี้ ราคา ไม่ เปลี่ยนแปลง จาก สัปดาห์ ที่ผ่านมา อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 25 กุมภาพันธ์ 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 350.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 1,007.75 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก เผชิญ ปัญหา ด้าน การคมนาคม ขนส่ง ผลผลิต ถั่วเหลือง จน เกิด การ ขาดแคลน ชั่วคราว",0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเนื่อง ด้าน ความต้องการ ใช้ มี ใกล้เคียง กับ ช่วง ที่ผ่านมา ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ทรงตัว อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 5 มีนาคม 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 334.40 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 979.25 เซนต์ / บุช เชล เป็น รา คาที่ ปรับ ลดลง เนื่องจาก ผลผลิต ถั่วเหลือง จาก อเมริกาใต้ มี มาก,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเนื่อง ด้าน ความต้องการ ใช้ มี ใกล้เคียง กับ ช่วง ที่ผ่านมา ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ทรงตัว อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 12 มีนาคม 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 343.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 986.50 เซนต์ / บุช เชล เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก บราซิล เผชิญ สภาพอากาศ ฝนตก ฉุก ส่งผล ให้การ เก็บเกี่ยว เกิด ความล่าช้า,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [17]:
x_c_vec.to_csv('/content/gdrive/My Drive/Colab Notebooks/x_c_vec.csv', index = False, header = True)

In [18]:
x_c_vec.iloc[1]

00        0
000       0
001       0
003       0
005       0
         ..
ไร        0
ไว        1
ไวร       0
ไอ        0
ไอโอวา    0
Name: สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหลือง ยังคง ทรงตัว   ขณะที่ ราคา ผลผลิต ถั่วเหลือง ในประเทศ ขึ้นอยู่กับ ราคา ของตลาด ชิคาโก เป็นหลัก   โดย กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า สัปดาห์ นี้ ราคา ไม่ เปลี่ยนแปลง จาก สัปดาห์ ที่ผ่านมา   อยู่ ที่ กิโลกรัม ละ   18.65   บาท   ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก   ประจำ วันที่   19   กุมภาพันธ์   2558   รอบ ส่งมอบ เดือน มีนาคม   ราคา อยู่ ที่   347.50   ดอลลาร์สหรัฐ / ชอร์ต ตัน   ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม   ราคา อยู่ ที่   1,007.25   เซนต์ / บุช เชล   ซึ่ง เป็น รา คาที่ ปรับ เพิ่มขึ้น   เนื่องจาก พื้นที่ ใน การ เพาะปลูก ถั่วเหลือง น้อยกว่า ที่ คาดการณ์ ไว้ ก่อนหน้านี้  , Length: 1543, dtype: int64

In [19]:
sum_c_vec = pd.DataFrame()

sum_c_vec['sum_c_vec'] = x_c_vec.iloc[:].sum(axis = 1)
sum_c_vec.reset_index(inplace = True)
sum_c_vec

,whole_word_token,sum_c_vec
0,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...,115
1,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,107
2,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,109
3,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,92
4,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,100
...,...,...
353,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,196
354,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,240
355,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,191
356,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,189


>>### Tf-idf Vec

>>>#### Whole

In [20]:
tfidf = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True)
tf_vec = tfidf.fit_transform(soybean['whole_word_token'])

In [21]:
x_tf_vec = pd.DataFrame(tf_vec.toarray(), columns=tfidf.get_feature_names(), index=soybean['whole_word_token'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
x_tf_vec.head()

,00,000,05,10,11,117,12,120,13,132,...,ในประเทศ,ในเกณฑ,ให,ใหญ,ใหม,ไป,ไม,ไร,ไว,ไวร
whole_word_token,,,,,,,,,,,,,,,,,,,,,
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกับ ความต้องการ ใช้ ที่ มี อย่าง ต่อเนื่อง ขณะที่ บราซิล ใกล้ เข้าสู่ ฤดูกาล เก็บเกี่ยว ผลผลิต ถั่วเหลือง ที่จะ มี ขึ้น ใน เดือน มีนาคม นี้ ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ลดลง จาก กิโลกรัม ละ 18.95 บาท เป็น กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 12 กุมภาพันธ์ 2557 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 330.50 ดอล ลา รส ห รัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 983.75 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ เพิ่มขึ้น เนื่องจาก บราซิล มี การคาดการณ์ ปี นี้ ผลผลิต ถั่วเหลือง มี น้อย,0.0,0.0,0.0,0.0,0.0,0.0,0.229315,0.0,0.0,0.0,...,0.000000,0.0,0.083000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
"สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหลือง ยังคง ทรงตัว ขณะที่ ราคา ผลผลิต ถั่วเหลือง ในประเทศ ขึ้นอยู่กับ ราคา ของตลาด ชิคาโก เป็นหลัก โดย กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า สัปดาห์ นี้ ราคา ไม่ เปลี่ยนแปลง จาก สัปดาห์ ที่ผ่านมา อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 19 กุมภาพันธ์ 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 347.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 1,007.25 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก พื้นที่ ใน การ เพาะปลูก ถั่วเหลือง น้อยกว่า ที่ คาดการณ์ ไว้ ก่อนหน้านี้",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.166647,0.0,0.000000,0.0,0.0,0.0,0.119271,0.0,0.186833,0.0
"สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหลือง ยังคง ทรงตัว ขณะที่ ราคา ผลผลิต ถั่วเหลือง ในประเทศ ขึ้นอยู่กับ ราคา ของตลาด ชิคาโก เป็นหลัก โดย กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า สัปดาห์ นี้ ราคา ไม่ เปลี่ยนแปลง จาก สัปดาห์ ที่ผ่านมา อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 25 กุมภาพันธ์ 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 350.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 1,007.75 เซนต์ / บุช เชล ซึ่ง เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก เผชิญ ปัญหา ด้าน การคมนาคม ขนส่ง ผลผลิต ถั่วเหลือง จน เกิด การ ขาดแคลน ชั่วคราว",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.157341,0.0,0.000000,0.0,0.0,0.0,0.112610,0.0,0.000000,0.0
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเนื่อง ด้าน ความต้องการ ใช้ มี ใกล้เคียง กับ ช่วง ที่ผ่านมา ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ทรงตัว อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 5 มีนาคม 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 334.40 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 979.25 เซนต์ / บุช เชล เป็น รา คาที่ ปรับ ลดลง เนื่องจาก ผลผลิต ถั่วเหลือง จาก อเมริกาใต้ มี มาก,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.111986,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเนื่อง ด้าน ความต้องการ ใช้ มี ใกล้เคียง กับ ช่วง ที่ผ่านมา ส่งผล ให้ราคา กาก ถั่วเหลือง ที่ ได้ จาก เมล็ด ถั่วเหลือง นำเข้า ทรงตัว อยู่ ที่ กิโลกรัม ละ 18.65 บาท ด้าน ราคา ซื้อขาย กาก ถั่วเหลือง ใน ตลาด ซื้อขาย ล่วงหน้า ชิคาโก ประจำ วันที่ 12 มีนาคม 2558 รอบ ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 343.50 ดอลลาร์สหรัฐ / ชอร์ต ตัน ส่วน ถั่วเหลือง ส่งมอบ เดือน มีนาคม ราคา อยู่ ที่ 986.50 เซนต์ / บุช เชล เป็น รา คาที่ ปรับ เพิ่มขึ้น เนื่องจาก บราซิล เผชิญ สภาพอากาศ ฝนตก ฉุก ส่งผล ให้การ เก็บเกี่ยว เกิด ความล่าช้า,0.0,0.0,0.0,0.0,0.0,0.0,0.243612,0.0,0.0,0.0,...,0.000000,0.0,0.149294,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [23]:
sum_tf_vec = pd.DataFrame()

sum_tf_vec['sum_tf_vec'] = x_tf_vec.iloc[:].sum(axis = 1)
sum_tf_vec.reset_index(inplace = True)
sum_tf_vec

,whole_word_token,sum_tf_vec
0,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...,6.063669
1,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,5.812786
2,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,5.753888
3,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,5.227960
4,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,5.703614
...,...,...
353,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,8.822123
354,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,8.600711
355,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,8.671932
356,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,8.306811


>## Join Soybean and vec

In [24]:
soybean = pd.concat([soybean, sum_c_vec['sum_c_vec'], sum_tf_vec['sum_tf_vec']], axis=1)
soybean

,url,title,ถั่วเหลือง,product,news_price,week,dates,whole_word_token,sum_c_vec,sum_tf_vec
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...,115,6.063669
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 16-20 กุมภาพันธ์ 2558,201508,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,107,5.812786
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 23 - 27กุมภาพันธ์2558,201509,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,109,5.753888
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 2 - 6 มีนาคม 2558,201510,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,92,5.227960
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 9-13 มีนาคม 2558,201511,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,100,5.703614
...,...,...,...,...,...,...,...,...,...,...
353,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 3-7 มกราคม 2565,202201,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,196,8.822123
354,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 10-14 มกราคม 2565,202202,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,240,8.600711
355,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 17-21 มกราคม 2565,202203,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,191,8.671932
356,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 24-28 มกราคม 2565,202204,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,189,8.306811


# Preprocessing soybean daily price from excel

>## Import data

In [25]:
price = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/soybean_price.csv')
price = price.rename(columns={'Dates': 'dates'})
price = price[['dates','PX_SETTLE']]
price

,dates,PX_SETTLE
0,3/1/2000,464.50
1,4/1/2000,471.50
2,5/1/2000,477.00
3,6/1/2000,475.50
4,7/1/2000,478.75
...,...,...
5779,25/2/2022,1590.25
5780,28/2/2022,1644.25
5781,1/3/2022,1690.00
5782,2/3/2022,1663.00


In [26]:
def get_data_from_years(dataframe, wanted_years):
  assert type(wanted_years) is list
  assert len(wanted_years) > 0

  idx = []
  for index, row in dataframe.iterrows():
    if int(str(row['dates'])[:4]) in wanted_years:
      idx.append(index)

  return pd.DataFrame(dataframe.iloc[idx]).reset_index(drop=True)

>## Group date

In [27]:
price['dates'] = price['dates'].apply(group_date)

>## Cal mean of soybean daily price

In [28]:
price_mean = price.groupby(['dates']).mean()
price_mean.reset_index(inplace = True)
price_mean = price_mean.rename(columns={'PX_SETTLE': 'avg_per_week'})

>## Classification price next week

In [29]:
price_mean['p_return'] = (price_mean['avg_per_week'].diff()/price_mean['avg_per_week'].shift(1))*100
# price_mean
def class_trend(p_return):
  if p_return > 0:
    return str(1)
  else : 
    return str(0)
  
price_mean['price_next_week'] = price_mean['p_return'].apply(class_trend).shift(-1)
price_mean = price_mean[['dates', 'avg_per_week', 'price_next_week']]
soybean = pd.merge(soybean, price_mean, on="dates")
soybean

,url,title,ถั่วเหลือง,product,news_price,week,dates,whole_word_token,sum_c_vec,sum_tf_vec,avg_per_week,price_next_week
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...,115,6.063669,979.90,1
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 16-20 กุมภาพันธ์ 2558,201508,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,107,5.812786,1000.10,1
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 23 - 27กุมภาพันธ์2558,201509,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,109,5.753888,1015.55,0
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 2 - 6 มีนาคม 2558,201510,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,92,5.227960,998.10,0
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 9-13 มีนาคม 2558,201511,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,100,5.703614,987.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
353,981458,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 3-7 มกร...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 3-7 มกราคม 2565,202201,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,196,8.822123,1387.50,0
354,982654,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 10-14 มก...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,1,วันที่ 10-14 มกราคม 2565,202202,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,240,8.600711,1383.50,1
355,983942,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 17-21 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 17-21 มกราคม 2565,202203,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,191,8.671932,1392.45,1
356,985320,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 24-28 ม...,กากถั่วเหลืองจากเมล็ดถั่วเหลืองนำเข้าในสัปดาห์...,soybean,0,วันที่ 24-28 มกราคม 2565,202204,กาก ถั่วเหลือง จาก เมล็ด ถั่วเหลือง นำเข้า ใน ...,189,8.306811,1433.70,1


# Preprocessing technical indicator

>>### Import technical indicator

In [30]:
soybean_tech = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/soybean_technical_indicator.csv')
soybean_tech = soybean_tech.rename(columns={'Dates': 'dates'})
soybean_tech

,dates,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,TAS_D,TAS_DS,TAS_DSS,CMCI,RSI,BB_MA,BB_UPPER,BB_LOWER,BB_WIDTH,BB_PERCENT
0,3/1/2000,-2.439241,-3.379858,0.940617,51.048951,37.062937,40.475524,40.242424,133.895134,51.180340,461.4375,478.357071,444.517929,7.333418,0.590502
1,4/1/2000,-1.386968,-2.981280,1.594312,70.629371,44.335664,40.783217,40.727273,137.229271,56.773607,461.3875,478.181453,444.593547,7.279761,0.801076
2,5/1/2000,-0.107985,-2.406621,2.298636,86.013986,54.825175,43.216783,41.491841,154.947252,60.593795,461.4125,478.297608,444.527392,7.318878,0.961575
3,6/1/2000,0.775639,-1.770169,2.545808,82.394366,66.688663,48.274796,44.091599,145.214258,59.060774,461.1750,477.099902,445.250098,6.906229,0.949767
4,7/1/2000,1.718358,-1.072464,2.790821,94.890511,76.995437,55.981575,49.157718,125.885690,61.342833,461.5125,478.585487,444.439513,7.398710,1.004818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5774,18/2/2022,52.368951,53.613129,-1.244179,87.425150,80.063685,80.898682,83.040517,120.196970,71.511297,1531.7750,1657.901712,1405.648288,16.468047,0.776409
5775,21/2/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5776,22/2/2022,54.113477,53.713199,0.400278,97.479839,84.008020,80.543804,81.415095,200.074897,75.341738,1543.3750,1662.472387,1424.277613,15.433370,0.884664
5777,23/2/2022,58.054477,54.581455,3.473023,98.425197,89.759849,82.210628,81.217705,248.988628,78.976536,1556.7625,1671.768204,1441.756796,14.774984,1.014051


>>### Group date

In [31]:
soybean_tech['dates'] = soybean_tech['dates'].apply(group_date)
soybean_tech

,dates,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,TAS_D,TAS_DS,TAS_DSS,CMCI,RSI,BB_MA,BB_UPPER,BB_LOWER,BB_WIDTH,BB_PERCENT
0,200001,-2.439241,-3.379858,0.940617,51.048951,37.062937,40.475524,40.242424,133.895134,51.180340,461.4375,478.357071,444.517929,7.333418,0.590502
1,200001,-1.386968,-2.981280,1.594312,70.629371,44.335664,40.783217,40.727273,137.229271,56.773607,461.3875,478.181453,444.593547,7.279761,0.801076
2,200001,-0.107985,-2.406621,2.298636,86.013986,54.825175,43.216783,41.491841,154.947252,60.593795,461.4125,478.297608,444.527392,7.318878,0.961575
3,200001,0.775639,-1.770169,2.545808,82.394366,66.688663,48.274796,44.091599,145.214258,59.060774,461.1750,477.099902,445.250098,6.906229,0.949767
4,200001,1.718358,-1.072464,2.790821,94.890511,76.995437,55.981575,49.157718,125.885690,61.342833,461.5125,478.585487,444.439513,7.398710,1.004818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5774,202207,52.368951,53.613129,-1.244179,87.425150,80.063685,80.898682,83.040517,120.196970,71.511297,1531.7750,1657.901712,1405.648288,16.468047,0.776409
5775,202208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5776,202208,54.113477,53.713199,0.400278,97.479839,84.008020,80.543804,81.415095,200.074897,75.341738,1543.3750,1662.472387,1424.277613,15.433370,0.884664
5777,202208,58.054477,54.581455,3.473023,98.425197,89.759849,82.210628,81.217705,248.988628,78.976536,1556.7625,1671.768204,1441.756796,14.774984,1.014051


>>### Cal mean of tech indicator

In [32]:
soybean_tech_mean = soybean_tech.groupby(['dates']).mean()
soybean_tech_mean.reset_index(inplace = True)
soybean_tech_focus = soybean_tech_mean
soybean_tech_focus

,dates,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,TAS_D,TAS_DS,TAS_DSS,CMCI,RSI,BB_MA,BB_UPPER,BB_LOWER,BB_WIDTH,BB_PERCENT
0,200001,-0.288039,-2.322078,2.034039,76.995437,55.981575,45.746379,43.142171,139.434321,57.790270,461.385000,478.104304,444.665696,7.247399,0.861548
1,200002,3.413095,0.780966,2.632130,87.276891,85.284123,77.209341,70.972132,113.630816,62.056944,464.785000,484.968344,444.601656,8.680666,0.932228
2,200003,8.664938,4.703977,3.960961,96.492049,94.328129,89.880325,87.958908,140.512916,72.731675,474.484375,504.094872,444.873878,12.476704,0.993560
3,200004,12.672807,8.989467,3.683340,95.561264,96.073149,95.800922,95.030973,118.007417,76.071027,486.045000,524.164873,447.925127,15.681148,0.926656
4,200005,11.384361,11.299670,0.084691,61.995457,76.655726,87.254364,90.702497,-24.055362,58.097989,498.455000,532.068461,464.841539,13.492254,0.614065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,202204,29.307491,28.483037,0.824454,87.770796,84.890752,81.018758,79.712178,125.491826,65.806404,1386.732500,1444.112373,1329.352627,8.272546,0.898299
1152,202205,45.743451,36.057292,9.686159,93.508507,93.537650,90.740584,88.964936,153.366502,78.144735,1421.562500,1518.714422,1324.410578,13.647321,1.083925
1153,202206,59.093875,49.514948,9.578927,89.584269,92.579411,93.382183,93.445005,95.489963,77.705119,1465.287500,1614.401619,1316.173381,20.345515,0.889471
1154,202207,54.138684,54.110790,0.027894,80.063685,80.898682,85.759345,88.020873,66.606267,69.305017,1513.472500,1650.009558,1376.935442,18.053600,0.746268


# Preprocessing fundamental indicator

>## Import funda indicator

In [33]:
soybean_funda = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/soybean_funda_indicator.csv')
soybean_funda = soybean_funda.rename(columns={'Dates': 'dates'})
soybean_funda.dropna(inplace = True)

In [34]:
soybean_funda

,dates,Unnamed: 0_y,Month,DatePublishKey,DateReferenceDataKey,Year,isLatest,ProductName,LocationName,Area Harvested_Million Acres,...,Ending Stocks_Million Bushels,Exports_Million Bushels,Imports_Million Bushels,Production_Million Bushels,Residual Use_Million Bushels,Seed Use_Million Bushels,Total demand_Million Bushels,Total Supply_Million Bushels,Yield_Bushels/Acre,Date
3904,12/1/2000,0.0,1.0,20000112.0,20000112.0,2000.0,1.0,Soybean,United States of America,72.5,...,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5,12.0
3926,11/2/2000,1.0,2.0,20000211.0,20000211.0,2000.0,1.0,Soybean,United States of America,72.5,...,345.0,890.0,3.0,2643.0,69.0,90.0,2649.0,2994.0,36.5,11.0
3946,10/3/2000,2.0,3.0,20000310.0,20000310.0,2000.0,1.0,Soybean,United States of America,72.5,...,325.0,910.0,3.0,2643.0,69.0,90.0,2669.0,2994.0,36.5,10.0
3968,11/4/2000,3.0,4.0,20000411.0,20000411.0,2000.0,1.0,Soybean,United States of America,72.5,...,305.0,930.0,3.0,2643.0,79.0,90.0,2689.0,2994.0,36.5,11.0
3991,12/5/2000,4.0,5.0,20000512.0,20000512.0,2000.0,1.0,Soybean,United States of America,73.9,...,495.0,970.0,3.0,2955.0,82.0,91.0,2763.0,3258.0,40.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9578,12/10/2021,259.0,10.0,20211012.0,20211012.0,2021.0,1.0,Soybean,United States of America,86.4,...,320.0,2090.0,15.0,4448.0,15.0,104.0,4399.0,4719.0,51.5,12.0
9598,9/11/2021,260.0,11.0,20211109.0,20211109.0,2021.0,1.0,Soybean,United States of America,86.4,...,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2,9.0
9620,9/12/2021,261.0,12.0,20211209.0,20211209.0,2021.0,1.0,Soybean,United States of America,86.4,...,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2,9.0
9644,12/1/2022,262.0,1.0,20220112.0,20220112.0,2022.0,1.0,"Oilseed, Soybean",United States,86.3,...,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4,12.0


>## Group date

In [35]:
soybean_funda['dates'] = soybean_funda['dates'].apply(group_date)
soybean_funda.reset_index(drop=True, inplace=True)

In [36]:
soybean_funda.columns

Index(['dates', 'Unnamed: 0_y', 'Month', 'DatePublishKey',
       'DateReferenceDataKey', 'Year', 'isLatest', 'ProductName',
       'LocationName', 'Area Harvested_Million Acres',
       'Area Planted_Million Acres', 'Average Farm Price_US Dollars/Bushel',
       'Begining Stocks_Million Bushels', 'Crushing_Million Bushels',
       'Ending Stocks_Million Bushels', 'Exports_Million Bushels',
       'Imports_Million Bushels', 'Production_Million Bushels',
       'Residual Use_Million Bushels', 'Seed Use_Million Bushels',
       'Total demand_Million Bushels', 'Total Supply_Million Bushels',
       'Yield_Bushels/Acre', 'Date'],
      dtype='object')

>## Select funda indicator

In [37]:
soybean_funda_focus = soybean_funda[['dates', 'Area Harvested_Million Acres',
       'Area Planted_Million Acres', 'Average Farm Price_US Dollars/Bushel',
       'Begining Stocks_Million Bushels', 'Crushing_Million Bushels',
       'Ending Stocks_Million Bushels', 'Exports_Million Bushels',
       'Imports_Million Bushels', 'Production_Million Bushels',
       'Residual Use_Million Bushels', 'Seed Use_Million Bushels',
       'Total demand_Million Bushels', 'Total Supply_Million Bushels',
       'Yield_Bushels/Acre']]
soybean_funda_focus

,dates,Area Harvested_Million Acres,Area Planted_Million Acres,Average Farm Price_US Dollars/Bushel,Begining Stocks_Million Bushels,Crushing_Million Bushels,Ending Stocks_Million Bushels,Exports_Million Bushels,Imports_Million Bushels,Production_Million Bushels,Residual Use_Million Bushels,Seed Use_Million Bushels,Total demand_Million Bushels,Total Supply_Million Bushels,Yield_Bushels/Acre
0,200002,72.5,73.8,4.75,348.0,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
1,200006,72.5,73.8,4.75,348.0,1600.0,345.0,890.0,3.0,2643.0,69.0,90.0,2649.0,2994.0,36.5
2,200010,72.5,73.8,4.70,348.0,1600.0,325.0,910.0,3.0,2643.0,69.0,90.0,2669.0,2994.0,36.5
3,200015,72.5,73.8,4.70,348.0,1590.0,305.0,930.0,3.0,2643.0,79.0,90.0,2689.0,2994.0,36.5
4,200019,73.9,74.9,4.50,300.0,1620.0,495.0,970.0,3.0,2955.0,82.0,91.0,2763.0,3258.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,202141,86.4,87.2,12.35,256.0,2190.0,320.0,2090.0,15.0,4448.0,15.0,104.0,4399.0,4719.0,51.5
260,202145,86.4,87.2,12.10,256.0,2190.0,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2
261,202149,86.4,87.2,12.10,256.0,2190.0,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2
262,202202,86.3,87.2,12.60,257.0,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4


# Join data

>## All data

In [38]:
soybean.head()

,url,title,ถั่วเหลือง,product,news_price,week,dates,whole_word_token,sum_c_vec,sum_tf_vec,avg_per_week,price_next_week
0,634817,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9 -13 ก...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีเพียงพอกับความต้...,soybean,-1,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี เพียง พอกั...,115,6.063669,979.90,1
1,635767,สรุปภาวะสินค้าเกษตรประจำสัปดาห์วันที่ 16-20 กุ...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 16-20 กุมภาพันธ์ 2558,201508,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,107,5.812786,1000.10,1
2,637001,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 23 - 27...,สัปดาห์นี้ความต้องการใช้ผลผลิตกากถั่วเหลืองยัง...,soybean,0,วันที่ 23 - 27กุมภาพันธ์2558,201509,สัปดาห์ นี้ ความต้องการ ใช้ ผลผลิต กาก ถั่วเหล...,109,5.753888,1015.55,0
3,638008,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 2 - 6 ม...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 2 - 6 มีนาคม 2558,201510,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,92,5.227960,998.10,0
4,639085,สรุปภาวะสินค้าเกษตรประจำสัปดาห์ วันที่ 9-13 มี...,ในสัปดาห์นี้ปริมาณถั่วเหลืองมีอย่างต่อเนื่อง ด...,soybean,0,วันที่ 9-13 มีนาคม 2558,201511,ใน สัปดาห์ นี้ ปริมาณ ถั่วเหลือง มี อย่าง ต่อเ...,100,5.703614,987.00,0


In [39]:
soybean_focus = soybean[['week', 'dates','price_next_week', 'sum_c_vec', 'sum_tf_vec', 'news_price']]
soybean_focus

,week,dates,price_next_week,sum_c_vec,sum_tf_vec,news_price
0,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,1,115,6.063669,-1
1,วันที่ 16-20 กุมภาพันธ์ 2558,201508,1,107,5.812786,0
2,วันที่ 23 - 27กุมภาพันธ์2558,201509,0,109,5.753888,0
3,วันที่ 2 - 6 มีนาคม 2558,201510,0,92,5.227960,0
4,วันที่ 9-13 มีนาคม 2558,201511,0,100,5.703614,0
...,...,...,...,...,...,...
353,วันที่ 3-7 มกราคม 2565,202201,0,196,8.822123,1
354,วันที่ 10-14 มกราคม 2565,202202,1,240,8.600711,1
355,วันที่ 17-21 มกราคม 2565,202203,1,191,8.671932,0
356,วันที่ 24-28 มกราคม 2565,202204,1,189,8.306811,0


>>### Tech

In [40]:
soybean_tech_focus

,dates,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,TAS_D,TAS_DS,TAS_DSS,CMCI,RSI,BB_MA,BB_UPPER,BB_LOWER,BB_WIDTH,BB_PERCENT
0,200001,-0.288039,-2.322078,2.034039,76.995437,55.981575,45.746379,43.142171,139.434321,57.790270,461.385000,478.104304,444.665696,7.247399,0.861548
1,200002,3.413095,0.780966,2.632130,87.276891,85.284123,77.209341,70.972132,113.630816,62.056944,464.785000,484.968344,444.601656,8.680666,0.932228
2,200003,8.664938,4.703977,3.960961,96.492049,94.328129,89.880325,87.958908,140.512916,72.731675,474.484375,504.094872,444.873878,12.476704,0.993560
3,200004,12.672807,8.989467,3.683340,95.561264,96.073149,95.800922,95.030973,118.007417,76.071027,486.045000,524.164873,447.925127,15.681148,0.926656
4,200005,11.384361,11.299670,0.084691,61.995457,76.655726,87.254364,90.702497,-24.055362,58.097989,498.455000,532.068461,464.841539,13.492254,0.614065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,202204,29.307491,28.483037,0.824454,87.770796,84.890752,81.018758,79.712178,125.491826,65.806404,1386.732500,1444.112373,1329.352627,8.272546,0.898299
1152,202205,45.743451,36.057292,9.686159,93.508507,93.537650,90.740584,88.964936,153.366502,78.144735,1421.562500,1518.714422,1324.410578,13.647321,1.083925
1153,202206,59.093875,49.514948,9.578927,89.584269,92.579411,93.382183,93.445005,95.489963,77.705119,1465.287500,1614.401619,1316.173381,20.345515,0.889471
1154,202207,54.138684,54.110790,0.027894,80.063685,80.898682,85.759345,88.020873,66.606267,69.305017,1513.472500,1650.009558,1376.935442,18.053600,0.746268


>>### Funda

In [41]:
soybean_funda_focus

,dates,Area Harvested_Million Acres,Area Planted_Million Acres,Average Farm Price_US Dollars/Bushel,Begining Stocks_Million Bushels,Crushing_Million Bushels,Ending Stocks_Million Bushels,Exports_Million Bushels,Imports_Million Bushels,Production_Million Bushels,Residual Use_Million Bushels,Seed Use_Million Bushels,Total demand_Million Bushels,Total Supply_Million Bushels,Yield_Bushels/Acre
0,200002,72.5,73.8,4.75,348.0,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
1,200006,72.5,73.8,4.75,348.0,1600.0,345.0,890.0,3.0,2643.0,69.0,90.0,2649.0,2994.0,36.5
2,200010,72.5,73.8,4.70,348.0,1600.0,325.0,910.0,3.0,2643.0,69.0,90.0,2669.0,2994.0,36.5
3,200015,72.5,73.8,4.70,348.0,1590.0,305.0,930.0,3.0,2643.0,79.0,90.0,2689.0,2994.0,36.5
4,200019,73.9,74.9,4.50,300.0,1620.0,495.0,970.0,3.0,2955.0,82.0,91.0,2763.0,3258.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,202141,86.4,87.2,12.35,256.0,2190.0,320.0,2090.0,15.0,4448.0,15.0,104.0,4399.0,4719.0,51.5
260,202145,86.4,87.2,12.10,256.0,2190.0,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2
261,202149,86.4,87.2,12.10,256.0,2190.0,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2
262,202202,86.3,87.2,12.60,257.0,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4


>## Merge tech & funda

In [42]:
soybean_tech_funda = pd.merge_ordered(soybean_tech_focus, soybean_funda_focus, on='dates', fill_method='ffill', how='left')
soybean_tech_funda

,dates,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,TAS_D,TAS_DS,TAS_DSS,CMCI,RSI,...,Crushing_Million Bushels,Ending Stocks_Million Bushels,Exports_Million Bushels,Imports_Million Bushels,Production_Million Bushels,Residual Use_Million Bushels,Seed Use_Million Bushels,Total demand_Million Bushels,Total Supply_Million Bushels,Yield_Bushels/Acre
0,200001,-0.288039,-2.322078,2.034039,76.995437,55.981575,45.746379,43.142171,139.434321,57.790270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200002,3.413095,0.780966,2.632130,87.276891,85.284123,77.209341,70.972132,113.630816,62.056944,...,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
2,200003,8.664938,4.703977,3.960961,96.492049,94.328129,89.880325,87.958908,140.512916,72.731675,...,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
3,200004,12.672807,8.989467,3.683340,95.561264,96.073149,95.800922,95.030973,118.007417,76.071027,...,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
4,200005,11.384361,11.299670,0.084691,61.995457,76.655726,87.254364,90.702497,-24.055362,58.097989,...,1605.0,365.0,865.0,3.0,2643.0,69.0,90.0,2629.0,2994.0,36.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,202204,29.307491,28.483037,0.824454,87.770796,84.890752,81.018758,79.712178,125.491826,65.806404,...,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4
1152,202205,45.743451,36.057292,9.686159,93.508507,93.537650,90.740584,88.964936,153.366502,78.144735,...,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4
1153,202206,59.093875,49.514948,9.578927,89.584269,92.579411,93.382183,93.445005,95.489963,77.705119,...,2215.0,325.0,2050.0,15.0,4435.0,16.0,102.0,4382.0,4707.0,51.4
1154,202207,54.138684,54.110790,0.027894,80.063685,80.898682,85.759345,88.020873,66.606267,69.305017,...,2215.0,325.0,2050.0,15.0,4435.0,16.0,102.0,4382.0,4707.0,51.4


>## Merge soybean & tech & funda

>>### Whole

In [43]:
soybean_dataset = pd.merge(soybean_focus, soybean_tech_funda, on="dates")
soybean_dataset

,week,dates,price_next_week,sum_c_vec,sum_tf_vec,news_price,MACD,MACD_SIGNAL,MACD_DIFF,TAS_K,...,Crushing_Million Bushels,Ending Stocks_Million Bushels,Exports_Million Bushels,Imports_Million Bushels,Production_Million Bushels,Residual Use_Million Bushels,Seed Use_Million Bushels,Total demand_Million Bushels,Total Supply_Million Bushels,Yield_Bushels/Acre
0,วันที่ 9 -13 กุมภาพันธ์ 2558,201507,1,115,6.063669,-1,-10.640456,-12.772359,2.131903,40.442610,...,1795.0,385.0,1790.0,25.0,3969.0,24.0,92.0,3701.0,4086.0,47.8
1,วันที่ 16-20 กุมภาพันธ์ 2558,201508,1,107,5.812786,0,-3.022899,-8.237728,5.214829,81.127346,...,1795.0,385.0,1790.0,25.0,3969.0,24.0,92.0,3701.0,4086.0,47.8
2,วันที่ 23 - 27กุมภาพันธ์2558,201509,0,109,5.753888,0,2.948086,-2.374953,5.323039,82.485961,...,1795.0,385.0,1790.0,25.0,3969.0,24.0,92.0,3701.0,4086.0,47.8
3,วันที่ 2 - 6 มีนาคม 2558,201510,0,92,5.227960,0,4.384361,2.591326,1.793035,45.347177,...,1795.0,385.0,1790.0,25.0,3969.0,24.0,92.0,3701.0,4086.0,47.8
4,วันที่ 9-13 มีนาคม 2558,201511,0,100,5.703614,0,-1.004628,1.145109,-2.149738,25.504981,...,1795.0,385.0,1790.0,25.0,3969.0,24.0,92.0,3701.0,4086.0,47.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,วันที่ 3-7 มกราคม 2565,202201,0,196,8.822123,1,30.709619,24.604390,6.105229,94.230787,...,2190.0,340.0,2050.0,15.0,4425.0,15.0,102.0,4356.0,4696.0,51.2
354,วันที่ 10-14 มกราคม 2565,202202,1,240,8.600711,1,32.286825,30.261029,2.025796,80.174787,...,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4
355,วันที่ 17-21 มกราคม 2565,202203,1,191,8.671932,0,26.811105,28.807734,-1.996629,81.269727,...,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4
356,วันที่ 24-28 มกราคม 2565,202204,1,189,8.306811,0,29.307491,28.483037,0.824454,87.770796,...,2190.0,350.0,2050.0,15.0,4435.0,16.0,102.0,4357.0,4707.0,51.4


In [44]:
soybean_dataset.to_csv('/content/gdrive/My Drive/Colab Notebooks/soybean_dataset.csv', index = False, header = True)